In [2]:
import numpy as np
import csv
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import ast
import tensorflow as tf
from keras.layers import LSTM

In [3]:
#defining labels

label_values = [0, 0.2] #<100% and >100% 

In [4]:
FEATURES = ["time", "price", "volume_from", "volume_to", "comments", "points", "followers", "fb_likes", "twitter_followers", "twitter_statuses", "reddit_subscribers", "reddit_active_users", "reddit_posts_per_day", "reddit_comments_per_day", "transaction_count", "large_transaction_count", "average_transaction_value", "unique_addresses_all_time", "current_supply"] #increase

'''
list = []
labels = []
for index, row in csv_data.iterrows():
    if (row["average"] != 0 and row["increase"] != "undefined"):
        tmp_list = []
        for i in range(0, len(FEATURES)):
            tmp_list.append(float(row[FEATURES[i]]))
        if (float(row["increase"]) > 100):
            labels.append(1)
        else:
            labels.append(0)
        list.append(tmp_list)
'''
f_x = open('X_filtered_not_regularized_data_2-1_week_v1.txt')
f_y = open('Y_filtered_not_regularized_data_2-1_week_v1.txt')
data_x = ast.literal_eval(f_x.read())
data_y = ast.literal_eval(f_y.read())

x = np.asarray(data_x)
y = np.asarray(data_y)
x, y = shuffle(x, y)

C:\Users\justi\AppData\Roaming\Python\Python37\site-packages\numpy\core\_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [66]:
full_data_x = []
full_data_y = []
for i in range(0, len(data_y)):
    full_data_y.append(data_y[i])

for i in range(0, len(data_x)):
    tmp_weeks_data = []

    for j in range(0, len(data_x[i])):
        tmp_day_data = []
        for k in range(0, len(data_x[i][j])):
            tmp_day_data.append(data_x[i][j][k])

        tmp_weeks_data.append(tmp_day_data)
    if (i<10): print(np.shape(np.asarray(tmp_weeks_data)))

    full_data_x.append(tmp_weeks_data)

(20, 19)
(20, 19)
(20, 19)
(20, 19)
(20, 19)
(20, 19)
(20, 19)
(20, 19)
(20, 19)
(20, 19)


ValueError: cannot reshape array of size 29973 into shape (29973,20,19)

In [100]:
full_data_x_2 = []

for i in range(0, len(data_x)):
    tmp_weeks_data = []

    for j in range(1, len(data_x[i])):
        tmp_day_data = []
        for k in range(0, len(data_x[i][j])):
            if j == 1:
                tmp_day_data.append(1)
            if (data_x[i][j-1][k] == 0 or data_x[i][j][k] == 0):
                if (data_x[i][j-1][k] == 0 and data_x[i][j][k] == 0):
                    tmp_day_data.append(1)
                elif (data_x[i][j-1][k] == 0):
                    tmp_day_data.append(data_x[i][j][k]/1)
                elif (data_x[i][j][k] == 0):
                    tmp_day_data.append(1/data_x[i][j-1][k])
            else:
                tmp_day_data.append(data_x[i][j][k]/data_x[i][j-1][k])

        tmp_weeks_data.append(tmp_day_data)
    full_data_x_2.append(tmp_weeks_data)

In [110]:
placeholder_data_x = data_x
full_data_x_zeros = np.zeros((len(data_x), 20, 19))
full_data_y_zeros = np.zeros((len(data_y)))

#for i in range(0, len(data_y)):
#    full_data_y.append(data_y[i])

for i in range(0, len(placeholder_data_x)):
    for j in range(0, len(placeholder_data_x[i])):
        for k in range(0, len(placeholder_data_x[i][j])):
            full_data_x_zeros[i][j][k] = placeholder_data_x[i][j][k]
for i in range(0, len(data_y)):
    full_data_y_zeros[i] = data_y[i]

In [82]:
#create x, y and shuffle them
x = full_data_x_zeros
y = full_data_y_zeros#np.asarray(data_y).astype(np.float32)
x, y = shuffle(x, y)
print(np.shape(x))
print(np.shape(y))

(29973, 20, 19)
(29973,)


In [83]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)
for i in range(0, len(y_train)):
    if y_train[i] >= label_values[1]:
        y_train[i] = 1
    else:
        y_train[i] = 0
for i in range(0, len(y_test)):
    if y_test[i] >= label_values[1]:
        y_test[i] = 1
    else:
        y_test[i] = 0

In [84]:
unique, counts = np.unique(y_train, return_counts=True)
unique_test, counts_test = np.unique(y_test, return_counts=True)

print("y_train: ", dict(zip(unique,counts)))
print("y_test: ", dict(zip(unique_test,counts_test)))
print(len(y_test))

y_train:  {0.0: 19820, 1.0: 2659}
y_test:  {0.0: 6522, 1.0: 972}
7494


In [89]:
model = Sequential()
#model.add(Dense(20, input_dim=, activation='relu'))
model.add(LSTM(128, input_shape=(20,19)))
model.add(Dense(8, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [90]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [91]:
model.fit(x_train, y_train, epochs=16, batch_size=64, validation_data=(x_test, y_test))

Epoch 1/16
352/352 [==============================] - 7s 15ms/step - loss: nan - accuracy: 0.8737 - val_loss: nan - val_accuracy: 0.8703
Epoch 2/16
352/352 [==============================] - 5s 14ms/step - loss: nan - accuracy: 0.8817 - val_loss: nan - val_accuracy: 0.8703
Epoch 3/16
352/352 [==============================] - 5s 14ms/step - loss: nan - accuracy: 0.8817 - val_loss: nan - val_accuracy: 0.8703 - accuracy: 0
Epoch 4/16
352/352 [==============================] - 5s 14ms/step - loss: nan - accuracy: 0.8817 - val_loss: nan - val_accuracy: 0.8703
Epoch 5/16
352/352 [==============================] - 5s 14ms/step - loss: nan - accuracy: 0.8817 - val_loss: nan - val_accuracy: 0.8703
Epoch 6/16
352/352 [==============================] - 5s 14ms/step - loss: nan - accuracy: 0.8817 - val_loss: nan - val_accuracy: 0.8703
Epoch 7/16
352/352 [==============================] - 5s 14ms/step - loss: nan - accuracy: 0.8817 - val_loss: nan - val_accuracy: 0.8703
Epoch 8/16
352/352 [=======